# Vectara 自主查询

[Vectara](https://vectara.com/) 是一家值得信赖的 AI 助手和代理平台，专注于为关键任务应用程序提供企业级就绪服务。
Vectara 无服务器 RAG 即服务通过易于使用的 API 提供了 RAG 的所有组件，包括：
1. 从文件（PDF、PPT、DOCX 等）提取文本的方法
2. 提供最先进性能的基于 ML 的分块。
3. [Boomerang](https://vectara.com/how-boomerang-takes-retrieval-augmented-generation-to-the-next-level-via-grounded-generation/) 嵌入模型。
4. 用于存储文本分块和嵌入向量的内部向量数据库。
5. 一个查询服务，可自动将查询编码为嵌入，并检索最相关的文本片段，包括对[混合搜索](https://docs.vectara.com/docs/api-reference/search-apis/lexical-matching)以及多种重新排序选项的支持，例如[多语言相关性重新排序器](https://www.vectara.com/blog/deep-dive-into-vectara-multilingual-reranker-v1-state-of-the-art-reranker-across-100-languages)、[MMR](https://vectara.com/get-diverse-results-and-comprehensive-summaries-with-vectaras-mmr-reranker/)、[UDF 重新排序器](https://www.vectara.com/blog/rag-with-user-defined-functions-based-reranking/)。
6. 一个 LLM，用于根据检索到的文档（上下文）创建[生成性摘要](https://docs.vectara.com/docs/learn/grounded-generation/grounded-generation-overview)，包括引用。

更多信息：
- [文档](https://docs.vectara.com/docs/)
- [API Playground](https://docs.vectara.com/docs/rest-api/)
- [快速入门](https://docs.vectara.com/docs/quickstart)

本 Notebook 展示了如何使用 `Vectara` 作为 `SelfQueryRetriever`。

## 设置

要使用 `VectaraVectorStore`，您首先需要安装相关软件包。


In [ ]:
!uv pip install -U pip && uv pip install -qU langchain-vectara

# 入门

开始之前，请按照以下步骤操作：
1. 如果您还没有账号，请[注册](https://www.vectara.com/integrations/langchain)免费试用 Vectara。
2. 在您的账户中，您可以创建一个或多个语料库。每个语料库代表一个在摄取输入文档时存储文本数据的区域。要创建语料库，请使用“**创建语料库**”按钮。然后为您的语料库提供一个名称和描述。您还可以选择定义过滤属性并应用一些高级选项。如果单击您创建的语料库，您可以在顶部看到其名称和语料库 ID。
3. 接下来，您需要创建 API 密钥来访问语料库。在语料库视图中单击“**访问控制**”选项卡，然后单击“**创建 API 密钥**”按钮。为您的密钥命名，并选择您想要查询专用权限还是查询+索引权限。单击“创建”，您现在就拥有一个活动的 API 密钥。请保密此密钥。

要将 LangChain 与 Vectara 结合使用，您需要获取这两个值：`corpus_key` 和 `api_key`。
您可以通过两种方式将 `VECTARA_API_KEY` 提供给 LangChain：

1. 在您的环境中包含这两个变量：`VECTARA_API_KEY`。

   例如，您可以使用 os.environ 和 getpass 进行如下设置：

```python
import os
import getpass

os.environ["VECTARA_API_KEY"] = getpass.getpass("Vectara API Key:")
```

2. 将它们添加到 `Vectara` vectorstore 构造函数中：

```python
vectara = Vectara(
    vectara_api_key=vectara_api_key
)
```

在本笔记本中，我们假设它们已在环境中提供。

## 从 LangChain 连接到 Vectara

在此示例中，我们假设您已创建帐户和语料库，并将 `VECTARA_CORPUS_KEY` 和 `VECTARA_API_KEY`（已授予索引和查询权限）设置为环境变量。

我们进一步假设该语料库定义了 4 个字段作为可过滤的元数据属性：`year`、`director`、`rating` 和 `genre`。

In [2]:
import os

from langchain_core.documents import Document

os.environ["VECTARA_API_KEY"] = "VECTARA_API_KEY"
os.environ["VECTARA_CORPUS_KEY"] = "VECTARA_CORPUS_KEY"

from langchain_vectara import Vectara

## 数据集

我们首先定义一个电影示例数据集，并将其与元数据一起上传到语料库：

In [6]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
        },
    ),
]

corpus_key = os.getenv("VECTARA_CORPUS_KEY")
vectara = Vectara()
for doc in docs:
    vectara.add_texts(
        [doc.page_content], corpus_key=corpus_key, doc_metadata=doc.metadata
    )

## 使用 Vectara 进行自查询
您无需通过 LangChain 机制进行自查询——在 Vectara 平台上启用 `intelligent_query_rewriting` 即可达到相同的结果。
Vectara 提供智能查询重写选项，该选项通过从自然语言查询自动生成元数据过滤器表达式来提高搜索精度。此功能分析用户查询，提取相关的元数据过滤器，并重写查询以专注于核心信息需求。更多[详情](https://docs.vectara.com/docs/search-and-retrieval/intelligent-query-rewriting)。

通过将 `VectaraQueryConfig` 中的 `intelligent_query_rewriting` 参数设置为 `true`，按每次查询启用智能查询重写。

In [7]:
from langchain_vectara.vectorstores import (
    CorpusConfig,
    SearchConfig,
    VectaraQueryConfig,
)

config = VectaraQueryConfig(
    search=SearchConfig(corpora=[CorpusConfig(corpus_key=corpus_key)]),
    generation=None,
    intelligent_query_rewriting=True,
)

## 查询
现在我们可以尝试实际使用我们的 vectara_queries 方法了！

In [8]:
# This example only specifies a relevant query
vectara.vectara_query("What are movies about scientists", config)

[(Document(metadata={'year': 1995, 'genre': 'animated', 'source': 'langchain'}, page_content='Toys come alive and have a blast doing so'),
  0.4141285717487335),
 (Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'source': 'langchain'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
  0.4046250879764557),
 (Document(metadata={'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2, 'source': 'langchain'}, page_content='Leo DiCaprio gets lost in a dream within a dream within a dream within a ...'),
  0.227469339966774),
 (Document(metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3, 'source': 'langchain'}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them'),
  0.19208428263664246),
 (Document(metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'source': 'langchain'}, page_content='A bunch of scientists bring back dinos

In [9]:
# This example only specifies a filter
vectara.vectara_query("I want to watch a movie rated higher than 8.5", config)

[(Document(metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, 'source': 'langchain'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
  0.34279149770736694),
 (Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'source': 'langchain'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
  0.242923304438591)]

In [12]:
# This example specifies a query and a filter
vectara.vectara_query("Has Greta Gerwig directed any movies about women", config)

[(Document(metadata={'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3, 'source': 'langchain'}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them'),
  0.10141132771968842)]

In [14]:
# This example specifies a composite filter
vectara.vectara_query("What's a highly rated (above 8.5) science fiction film?", config)

[(Document(metadata={'year': 1979, 'rating': 9.9, 'director': 'Andrei Tarkovsky', 'genre': 'science fiction', 'source': 'langchain'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
  0.9508692026138306)]

In [15]:
# This example specifies a query and composite filter
vectara.vectara_query(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated",
    config,
)

[(Document(metadata={'year': 1995, 'genre': 'animated', 'source': 'langchain'}, page_content='Toys come alive and have a blast doing so'),
  0.7290377616882324),
 (Document(metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'source': 'langchain'}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
  0.4838160574436188)]